# Project 3


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="moviegenre.png"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

• The project must be carried out in the groups assigned for module 4.
• Use clear and rigorous procedures.
• The delivery of the project is on July 12, 2020, 11:59 pm, through Sicua + (Upload: the API and the report in PDF format).
• No projects will be received after the delivery time or by any other means than the one established. 




### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [81]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [82]:
dataTraining = pd.read_csv('datasets/dataTraining.csv', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('datasets/dataTesting.csv', encoding='UTF-8', index_col=0)

In [83]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [84]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


### Create count vectorizer


In [85]:
vect = CountVectorizer(max_features=5000, lowercase=True, stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 5000)

In [86]:
print(vect.get_feature_names()[:50])

['aaron', 'abandon', 'abandoned', 'abby', 'abducted', 'abe', 'abigail', 'abilities', 'ability', 'able', 'aboard', 'absence', 'absolutely', 'abuse', 'abused', 'abusive', 'academic', 'academy', 'accept', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accident', 'accidental', 'accidentally', 'accompanied', 'accompany', 'accomplish', 'according', 'account', 'accountant', 'accused', 'ace', 'achieve', 'achieving', 'acquaintance', 'acquaintances', 'act', 'acting', 'action', 'actions', 'active', 'activist', 'activities', 'activity', 'actor', 'actors', 'actress']


### Create y

In [87]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [88]:
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

In [89]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.3, random_state=666)

### Train multi-class multi-label model

In [90]:
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=1000, max_depth=20, random_state=666, bootstrap=True))

In [91]:
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=20,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                              

In [93]:
y_pred_genres = clf.predict_proba(X_test)

In [94]:
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.8302188715841989

### Predict the testing dataset

In [95]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = clf.predict_proba(X_test_dtm)


In [96]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [97]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.106588,0.091338,0.021865,0.029514,0.376928,0.116531,0.030724,0.513936,0.059947,0.065514,...,0.022460,0.058365,0.000050,0.312965,0.048691,0.012532,0.013496,0.172860,0.017591,0.013758
4,0.113844,0.094361,0.022843,0.045524,0.321967,0.165270,0.034417,0.550499,0.068469,0.059467,...,0.019727,0.058633,0.000397,0.158345,0.048442,0.006002,0.015120,0.193362,0.023796,0.013547
5,0.171995,0.112100,0.022352,0.090874,0.265742,0.483691,0.032740,0.651209,0.087354,0.083917,...,0.030502,0.261720,0.000060,0.276785,0.070312,0.019299,0.028799,0.437291,0.030422,0.019231
6,0.145665,0.094412,0.021922,0.049061,0.277727,0.122885,0.049814,0.608170,0.063023,0.086744,...,0.037223,0.086391,0.000017,0.242480,0.089068,0.005820,0.024144,0.298621,0.060756,0.018048
7,0.183791,0.135110,0.021054,0.034805,0.362454,0.242562,0.031006,0.382868,0.075649,0.136196,...,0.035155,0.118254,0.000043,0.182180,0.401973,0.016600,0.014321,0.290829,0.017141,0.015429


In [98]:
res.to_csv('pred_genres_text_RF.csv', index_label='ID')